필요한 라이브러리 설치 및 데이터 불러오기


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install konlpy


In [1]:
import numpy as np
import pandas as pd
import csv

import matplotlib.pyplot as plt
import seaborn as sns

from konlpy.tag import *
from collections import Counter
from transformers import AutoModel, AutoTokenizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf
import torch

In [3]:
# 파일 가져오기
df_file = "C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/최종_데이터셋7(월,일,색상,RGB).csv"

df = pd.read_csv(df_file) #, encoding='CP949'
df.drop(["일자", "선택한 색상", "중앙 색상들"], axis=1, inplace=True)

NLP 처리 및 데이터 전처리


In [4]:
# NLP 처리 할 데이터만 가져오기
df_nlp = df[["꽃", "꽃말", "설명", "색상"]]
print(df_nlp.shape)
df_nlp.head(5)

# 꽃말, 설명 합치기 (색상은 유지)
df_nlp['설명'] = df_nlp['꽃말'] + ". " + df_nlp['설명']

# 특수문자 및 띄어쓰기 제거 함수
import re

def remove_special_characters(text):
    if not isinstance(text, str):
        text = str(text)

    # 특수문자 제거
    pattern = r'[^\w\s\.]' # 문자, 공백문자, 마침표 제외 제거
    clean_text = re.sub(pattern, '', text)

    # 한자 제거
    pattern = r'[\u4e00-\u9fff]' # 중국어 한자의 유니코드 시작과 끝 제거
    clean_text = re.sub(pattern, '', clean_text)
    clean_text = ' '.join(clean_text.split())
    return clean_text

df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)
df_nlp['색상'] = df_nlp['색상'].apply(remove_special_characters)

(837, 4)


C:\Users\pc\AppData\Local\Temp\ipykernel_19876\2385004683.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['꽃말'] + ". " + df_nlp['설명']
C:\Users\pc\AppData\Local\Temp\ipykernel_19876\2385004683.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)
C:\Users\pc\AppData\Local\Temp\ipykernel_19876\2385004683.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

Klue-RoBERTa 모델 및 토크나이저 불러오기


In [5]:
# Klue-RoBERTa 모델과 토크나이저 불러오기
model_name = 'klue/roberta-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# [CLS]와 [SEP] 토큰을 추가하는 함수
def add_special_tokens(text):
    # 문장을 마침표를 기준으로 분리
    sentences = text.split('. ')
    # 각 문장에 [CLS]와 [SEP] 토큰 추가
    sentences_with_special_tokens = ['[CLS] ' + sentence + ' [SEP]' for sentence in sentences]
    # 다시 문장으로 결합
    text_with_special_tokens = ' '.join(sentences_with_special_tokens)
    return text_with_special_tokens

df_nlp['설명2'] = df_nlp['설명'].apply(add_special_tokens)
df_nlp['색상2'] = df_nlp['색상'].apply(add_special_tokens)
df_nlp.head(10)


Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\pc\AppData\Local\Temp\ipykernel_19876\3946500010.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명2'] = df_nlp['설명'].apply(add_special_tokens)
C:\Users\pc\AppData\Local\Temp\ipykernel_19876\3946500010.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

,꽃,꽃말,설명,색상,설명2,색상2
0,각시붓꽃,"부끄러움, 세련됨",부끄러움 세련됨. 각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 ...,보라,[CLS] 부끄러움 세련됨 [SEP] [CLS] 각시라 하면 이제 막 시집 온 새색...,[CLS] 보라 [SEP]
1,감국,그윽한 향기,그윽한 향기. 가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인...,노랑,[CLS] 그윽한 향기 [SEP] [CLS] 가을 산야는 국화과 식물들 차지다 [S...,[CLS] 노랑 [SEP]
2,개나리,"희망,깊은 정, 달성",희망깊은 정 달성. 우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃 개...,노랑,[CLS] 희망깊은 정 달성 [SEP] [CLS] 우리나라 전역에서 봄 소식을 가장...,[CLS] 노랑 [SEP]
3,개나리,희망,희망. 개나리 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. 나리나리 개나리...,노랑,[CLS] 희망 [SEP] [CLS] 개나리 봄을 알리는 전령사 하면 가장 먼저 떠...,[CLS] 노랑 [SEP]
4,갯개미취,추억,추억. 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 ...,보라,[CLS] 추억 [SEP] [CLS] 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가...,[CLS] 보라 [SEP]
5,거베라,"신비, 수수께끼",신비 수수께끼. 거베라만큼 크고 화려한 꽃도 드물다. 부를 상징하기도 한다. 거베라...,분홍,[CLS] 신비 수수께끼 [SEP] [CLS] 거베라만큼 크고 화려한 꽃도 드물다 ...,[CLS] 분홍 [SEP]
6,검은포플라,용기,용기. 당신은 용기가 있어 주위 사람들도 당신을 의지하고 있습니다. 그렇다고 해서 ...,빨강,[CLS] 용기 [SEP] [CLS] 당신은 용기가 있어 주위 사람들도 당신을 의지...,[CLS] 빨강 [SEP]
7,겹 공작초,첫눈에 반함,첫눈에 반함. 자기가 가장 아름답다고 믿는 세 여신이 있었다. 제우스의 아내 헤라 ...,보라,[CLS] 첫눈에 반함 [SEP] [CLS] 자기가 가장 아름답다고 믿는 세 여신이...,[CLS] 보라 [SEP]
8,겹 과꽃,나의 사랑은 당신보다깊다,나의 사랑은 당신보다깊다. 괴테의 희곡 파우스트 에 등장하는 소녀 마가렛이 꽃잎을 ...,보라,[CLS] 나의 사랑은 당신보다깊다 [SEP] [CLS] 괴테의 희곡 파우스트 에 ...,[CLS] 보라 [SEP]
9,겹 소국,청초,청초. 국화 품종 중 작은 화형을 가진 국화를 소국이라고 하며 스프레이 형태를 띄고...,분홍,[CLS] 청초 [SEP] [CLS] 국화 품종 중 작은 화형을 가진 국화를 소국이...,[CLS] 분홍 [SEP]


벡터화 함수


In [6]:
# 문장을 벡터화 변환하는 함수
def get_sentence_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True) # 최대 길이 초과 시 잘라내기, 작은 경우 패딩 진행
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy() # 각 토큰 벡터의 첫 번째 벡터 확인 (CLS 토큰 벡터)

# 설명 벡터화
df_nlp['설명_벡터'] = df_nlp['설명2'].apply(lambda x: get_sentence_embedding(x, tokenizer, model).tolist()) # 데이터마다 (1, 768) 차원 벡터 생성
df_nlp['색상_벡터'] = df_nlp['색상2'].apply(lambda x: get_sentence_embedding(x, tokenizer, model).tolist())


월/계절 원핫인코딩


In [7]:
# 월/계절 원핫인코딩
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(df[['월', '계절']]).toarray()  # '월'과 '계절' 컬럼을 원핫인코딩
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['월', '계절'])) # 원핫인코딩된 결과를 새로운 데이터프레임으로 변환

# 원핫인코딩 결과를 '원핫인코딩' 컬럼으로 추가
df_nlp['원핫인코딩'] = encoded_df.apply(lambda row: list(row), axis=1)
df_nlp['원핫인코딩'] = df_nlp['원핫인코딩'].apply(lambda x: np.array(x).reshape(1, -1)) #(16,1)을 (1,16)차원으로 변경


결합 벡터 생성 함수


In [8]:
# 결합 벡터 생성 (설명 벡터, 색상 벡터, 원핫인코딩 벡터 결합)
def create_combined_vector(row):
    설명_벡터 = row['설명_벡터']
    색상_벡터 = row['색상_벡터']
    원핫인코딩 = row['원핫인코딩']

    combined_array = np.concatenate((설명_벡터, 색상_벡터, 원핫인코딩), axis=1)

    return combined_array

df_nlp['최종_벡터'] = df_nlp.apply(create_combined_vector, axis=1)

# 색상이 없는 경우 결합 벡터 생성 함수
def create_combined_vector_without_color(row):
    설명_벡터 = row['설명_벡터']
    원핫인코딩 = row['원핫인코딩']
    combined_array = np.concatenate((설명_벡터, 원핫인코딩), axis=1)
    return combined_array


In [10]:
df_nlp.head(5)

,꽃,꽃말,설명,색상,설명2,색상2,설명_벡터,색상_벡터,원핫인코딩,최종_벡터
0,각시붓꽃,"부끄러움, 세련됨",부끄러움 세련됨. 각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 ...,보라,[CLS] 부끄러움 세련됨 [SEP] [CLS] 각시라 하면 이제 막 시집 온 새색...,[CLS] 보라 [SEP],"[[0.07368195801973343, 0.15451766550540924, 0....","[[0.15296272933483124, 0.3076050877571106, -0....","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.07368195801973343, 0.15451766550540924, 0...."
1,감국,그윽한 향기,그윽한 향기. 가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인...,노랑,[CLS] 그윽한 향기 [SEP] [CLS] 가을 산야는 국화과 식물들 차지다 [S...,[CLS] 노랑 [SEP],"[[0.08965592086315155, 0.2825712561607361, 0.1...","[[0.11612437665462494, 0.3139854669570923, -0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.08965592086315155, 0.2825712561607361, 0.1..."
2,개나리,"희망,깊은 정, 달성",희망깊은 정 달성. 우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃 개...,노랑,[CLS] 희망깊은 정 달성 [SEP] [CLS] 우리나라 전역에서 봄 소식을 가장...,[CLS] 노랑 [SEP],"[[0.13718879222869873, 0.23945458233356476, 0....","[[0.11612437665462494, 0.3139854669570923, -0....","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.13718879222869873, 0.23945458233356476, 0...."
3,개나리,희망,희망. 개나리 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. 나리나리 개나리...,노랑,[CLS] 희망 [SEP] [CLS] 개나리 봄을 알리는 전령사 하면 가장 먼저 떠...,[CLS] 노랑 [SEP],"[[-0.06287881731987, 0.3564964234828949, 0.256...","[[0.11612437665462494, 0.3139854669570923, -0....","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.06287881731987, 0.3564964234828949, 0.256..."
4,갯개미취,추억,추억. 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 ...,보라,[CLS] 추억 [SEP] [CLS] 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가...,[CLS] 보라 [SEP],"[[0.17346309125423431, 0.0834614634513855, 0.0...","[[0.15296272933483124, 0.3076050877571106, -0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...","[[0.17346309125423431, 0.0834614634513855, 0.0..."


In [17]:
df_nlp['색상'] = df['색상']

In [18]:
df_nlp.to_csv("C:/Users/pc/Model/Model-7/dataset/추천시스템_데이터.csv", index=False)

사용자 입력 텍스트 분석 함수


In [9]:
# 사용자 입력 텍스트 분석 함수(월, 계절)
def extract_month_season(text):
    months = {
        '1월': 1, '2월': 2, '3월': 3, '4월': 4, '5월': 5, '6월': 6,
        '7월': 7, '8월': 8, '9월': 9, '10월': 10, '11월': 11, '12월': 12
    }
    seasons = {'봄': '봄', '여름': '여름', '가을': '가을', '겨울': '겨울'}

    month = None
    season = None

    for key, value in months.items():
        if key in text:
            month = value
            break

    for key in seasons.keys():
        if key in text:
            season = key
            break

    return month, season

# 확장된 색상 사전
color_synonyms = {
    '갈색': ['갈색', '브라운', '갈색의', '브라운색', '갈색이', '갈색이다', '갈색을', '갈색으로'],
    '노랑': ['노랑', '노란', '노란색', '황색', '노랑색', '노랗다', '노랗게', '노란빛', '노란 빛','누런'],
    '보라': ['보라', '보라색', '자주색', '보라빛', '보랏빛', '보라빛의', '보라빛이', '보라빛으로'],
    '분홍': ['분홍', '핑크', '분홍색', '핑크색', '분홍빛', '분홍빛의', '분홍빛이', '분홍빛으로'],
    '빨강': ['빨강', '빨간', '빨강색', '빨간색', '붉은', '붉은색', '붉은 빛', '붉다', '붉게'],
    '주황': ['주황', '주황색', '오렌지', '오렌지색', '주황빛', '주황빛의', '주황빛이', '주황빛으로'],
    '초록': ['초록', '초록색', '녹색', '초록빛', '초록의', '초록이', '초록으로'],
    '파랑': ['파랑', '파란', '파란색', '파랑색', '청색', '파랑빛', '파란빛', '파랗다', '파랗게','푸른','푸른빛', '푸른 색'],
    '흰색': ['흰색', '하양', '하얀', '백색', '하얀색', '하양색', '하얗다', '하얗게', '백색의', '백색이', '백색으로','흰']
}

# 입력 텍스트에서 색상을 추출하는 함수
def extract_color(text):
    text = text.lower()
    for color, synonyms in color_synonyms.items():
        for synonym in synonyms:
            if synonym in text:
                return color
    return None


사용자 입력을 벡터화 하는 함수


In [1]:
# 사용자 입력을 최종 벡터화
def get_user_input_vector(user_input, user_month):
    input_text = remove_special_characters(user_input)

    # 토큰화 및 벡터화
    input_embeddings = get_sentence_embedding(input_text, tokenizer, model)
    input_color_text = extract_color(user_input)
    month, season = extract_month_season(user_input)

    # 원핫 벡터 생성
    user_onehot_vector = np.zeros(len(encoder.get_feature_names_out(['월', '계절'])))
    if month is not None:
        month = user_month
        month_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'월_{month}')
        user_onehot_vector[month_idx] = 1
    if season is None and month is not None: #사용자가 입력한 월 기준으로 계절 추출
        if month in [3, 4, 5]:
            season = '봄'
        elif month in [6, 7, 8]:
            season = '여름'
        elif month in [9, 10, 11]:
            season = '가을'
        else:
            season = '겨울'
        season_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'계절_{season}')
        user_onehot_vector[season_idx] = 1
    
    if month is not None: #텍스트에 입력된 월
        month_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'월_{month}')
        user_onehot_vector[month_idx] = 1
    if season is not None: #텍스트에 입력된 계절
        season_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'계절_{season}')
        user_onehot_vector[season_idx] = 1
    user_onehot_vector = user_onehot_vector.reshape(1, -1) #(1,16)

    # 벡터 결합
    if input_color_text:
        color_embeddings = get_sentence_embedding(input_color_text, tokenizer, model)
        user_vector = np.concatenate((input_embeddings, color_embeddings, user_onehot_vector), axis=1)
    else:
        user_vector = np.concatenate((input_embeddings, user_onehot_vector), axis=1)

    return user_vector

형태소 확인

In [11]:
from konlpy.tag import Okt

# 형태소 분석기 초기화
okt = Okt()

# 입력 텍스트의 형태소 분석 및 명사형 확인 함수
def is_noun_phrase(text):
    # 형태소 분석
    pos_tags = okt.pos(text)
    # 디버깅을 위해 형태소 분석 결과 출력
    print(f"Text: {text}, POS Tags: {pos_tags}")
    # 모든 단어가 명사인지 확인
    for word, tag in pos_tags:
        if tag != 'Noun':
            return False
    return True

# 명사형 텍스트에 문맥 추가 함수
def add_context_if_noun(user_input):
    if is_noun_phrase(user_input):
        return user_input + "에 어울리는 꽃을 추천해." # " 꽃을 추천해"
    return user_input

특정 이벤트에 가중치를 부여하는 함수


In [53]:
# 특정 이벤트에 가중치를 부여하는 함수 (각 행마다 개별 적용)
def apply_event_weight_for_row(user_input, row):
    event_weights = {
        '발렌타인': 1.2,
        '화이트': 1.2,
        '어버이날': 1.2,
        '어버이': 1.2,
        '부모': 1.2,
        '부모님': 1.2,
        '성년의날': 1.2,
        '성년': 1.2,
        '로즈데이': 1.2,
        '로즈': 1.2,
        '스승의날': 1.2,
        '선생': 1.2,
        '스승': 1.2,
        '선생님': 1.2,
        '연인': 1.2,
        '여자친구': 1.2,
        '남자친구': 1.2,
        '여친': 1.2,
        '남친': 1.2,
        '아내': 1.2,
        '와이프': 1.2,
        '남편': 1.2,
        '생일': 1.2,
        '기념일': 1.2,
        '사랑': 1.2
    }
    weight = 1.0

    for event, event_weight in event_weights.items():
        if event in user_input:
            weight *= event_weight
            break

    #사용자 월 추출
    month, season = extract_month_season(user_input)

    # 꽃의 설명 및 월,계절에 따른 추가 가중치 적용
    if any(event in row['설명'] for event in event_weights.keys() if event in user_input):
        weight *= 1.2
    # 월에 따른 추가 가중치 적용
    if month and isinstance(row['월'], int) and month == row['월']:
        weight *= 1.2
    # 계절에 따른 추가 가중치 적용
    if season and isinstance(row['계절'], str) and season == row['계절']:
        weight *= 1.2

    return weight

추천 시스템 함수


In [54]:
# 추천 시스템 함수 (코사인 유사도 기반)
def recommend_flower(user_input, user_month=None):
    user_input = add_context_if_noun(user_input)
    user_vector = get_user_input_vector(user_input, user_month)
    input_color = extract_color(user_input)
    df = df_nlp

    if input_color:
        # 색상이 명시된 경우 해당 색상의 꽃들로 필터링
        filtered_df = df[df['색상'] == input_color]
        filtered_df['최종_벡터'] = filtered_df.apply(lambda row: create_combined_vector(row), axis=1)
    else:
        # 색상이 명시되지 않은 경우 결합 벡터에서 색상 벡터를 제외
        filtered_df = df.copy()
        filtered_df['최종_벡터'] = filtered_df.apply(lambda row: create_combined_vector_without_color(row), axis=1)

    # 각 행에 대해 가중치 계산 및 유사도 산출
    filtered_df['유사도'] = filtered_df.apply(lambda row: cosine_similarity(user_vector, np.array(row['최종_벡터']).reshape(1, -1))[0][0] * apply_event_weight_for_row(user_input, row), axis=1)
    filtered_df['유사도'] = filtered_df['유사도'].astype(float)  # 숫자형으로 변환

    # 유사도를 기준으로 상위 3개의 꽃을 선택하고 중복된 꽃을 제거
    top3 = filtered_df.nlargest(3, '유사도').drop_duplicates(subset='꽃')

    # 만약 중복 제거 후 3개의 꽃이 되지 않는 경우, 다시 nlargest로 채우기
    if top3.shape[0] < 3:
        additional_top = filtered_df.nlargest(20, '유사도')  # 상위 20개 정도를 선택
        additional_top = additional_top[~additional_top['꽃'].isin(top3['꽃'])]
        top3 = pd.concat([top3, additional_top]).nlargest(3, '유사도').drop_duplicates(subset='꽃')

    return top3[['꽃', '꽃말', '유사도']].to_dict('records')

In [55]:
user_input = "5월의 봄 어버이날"
recommendation = recommend_flower(user_input, 7)
recommendation

Text: 5월의 봄 어버이날, POS Tags: [('5월', 'Number'), ('의', 'Foreign'), ('봄', 'Noun'), ('어버이날', 'Noun')]


,꽃,꽃말,색상,유사도
661,카네이션,"여성의 애정, 모정",빨강,1.871463
361,뻐꾹채,나그네,보라,1.794261
36,괭이밥,빛나는 마음,초록,1.559836


테스트 예시


In [58]:
correct_file = "C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/추천시스템_데이터_평가.csv"

#답안지 불러오기
correct = pd.read_csv(correct_file, encoding='CP949')
correct.head(5)

,Unnamed: 0,input,output
0,0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,"['감국', '구절초', '국화', '꽃무릇(석산)', '꽃아그배나무꽃', '꽃향유..."
1,1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,"['꽃 아로니아', '꽃 영춘화', '꽃아까시나무꽃', '처녀치마', '철쭉']"
2,2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,"['거베라', '겹 소국', '겹 스토크', '겹 프릴 리시안셔스', '겹복숭아 꽃..."
3,3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,"['겹설유화', '공작초', '구즈마니아', '군자란', '꽃동백', '남천', '..."
4,4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,"['겹 알스트로메리아', '골담초', '까실쑥부쟁이', '깽깽이풀', '꽃동백', ..."


In [59]:
user_input=correct['input']
user_input = user_input.tolist()

In [17]:
'''user_input = [
    '어버이날',
    '어버이날에 어울리는 꽃을 추천해.'
]'''

user_input = [
    "여자친구 생일이 6월 10일이야",
    "친구가 어제 드디어 취업을 해서 꽃을 선물하려해",
    "어제 여자친구랑 싸워서 화해를 하려해",
    "부모님 생신이여서 꽃을 선물하고 싶은데 화려한 꽃으로 축하하고 싶어",
    "어버이날",
    "나는 1월에 겨울에 어울리는 용기 있는 꽃을 찾고 있어"
]


In [60]:
user_input

['스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.',
 '친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?',
 '응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.',
 '발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?',
 '친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?',
 '사랑을 전할 꽃 추천해줘. 특히 빨강 꽃을 좋아해.',
 '크리스마스에 선물할 수 있는 꽃을 추천해줘.',
 '하얀 꽃 추천해줘. 이번 달은 4월이야.',
 '자원봉사자들에게 겨울에 감사의 마음을 전할 꽃은 무엇이 좋을까?',
 '여름 이사 선물로 줄 만한 꽃 좀 추천해줘.',
 '동료가 승진했어, 축하할 꽃 추천해줘. 이번 달은 4월이야.',
 '부모님 결혼기념일이 겨울인데 어떤 꽃을 드리면 좋을까?',
 '분홍색 꽃 추천해줄래?',
 '보라 꽃 뭐가 있을까? 이번 달은 10월이야.',
 '동료의 생일이 다가와. 꽃 선물을 할까 하는데 추천 좀 해줘. 이번 달은 3월이야.',
 '사랑을 고백할 때 줄 꽃 뭐가 좋을까? 이번 달은 11월이야.',
 '감사를 전할 때 좋은 꽃 추천해줘. 특히 노랑 꽃을 좋아해.',
 '여자친구 생일인데 어떤 꽃이 좋을까? 특히 하양 꽃을 좋아해.',
 '요새 봄인데 피곤한 친구에게 활력을 줄 꽃은 어떤 게 좋을까?',
 '가을에 피는 꽃 좀 추천해줘. 특히 파랑 꽃을 좋아해.',
 '5월에 고백할 때 줄 꽃 추천해줘',
 '1월 기념일 선물로 좋은 꽃 뭐가 있어?',
 '사랑을 고백할 때 줄 꽃 뭐가 좋을까? 이번 달은 7월이야.',
 '5월 어버이날 선물로 꽃을 주고 싶은데 뭐가 좋을까?',
 '여자친구가 꽃을 좋아해, 무슨 꽃을 선물할까? 이번 달은 3월이야.',
 '결혼식에 어울리는 꽃 좀 추천해줘. 요즘 여름이야.',
 '부모님 결혼기념일이 7월로 곧인데 어떤 꽃을 드리면 좋을까?',
 '여름에 어울리는 꽃 뭐야? 특히 분홍 꽃을 좋아해.',
 '크리스마스에 장식할 꽃 추천해줄래? 특히 노랑 꽃을 좋아해.',
 

In [62]:
results = []

In [63]:
# 각 문장에 대해 추천 꽃을 구하고 결과 리스트에 추가
for row in user_input:
    recommendation = recommend_flower(row, df_nlp, tokenizer, model)  # 함수를 호출할 때 필요한 인자를 추가
    if recommendation.shape[0] > 0:
        # 추천 결과가 존재할 경우 각 꽃의 이름을 추출하여 리스트에 추가
        recommended_flowers = recommendation['꽃'].tolist()
        # 만약 추천된 꽃이 3개 미만일 경우 나머지는 '추천 없음'으로 채움
        recommended_flowers += ['추천 없음'] * (3 - len(recommended_flowers))
        results.append([row] + recommended_flowers)
    else:
        # 추천 결과가 없을 경우 '추천 없음'으로 표시
        results.append([row] + ['추천 없음', '추천 없음', '추천 없음'])

# 데이터 프레임으로 변환
df_results = pd.DataFrame(results, columns=["user_input", "추천꽃1", "추천꽃2", "추천꽃3"])

# 결과 출력
df_results

Text: 스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야., POS Tags: [('스승', 'Noun'), ('의', 'Josa'), ('날', 'Noun'), ('에', 'Josa'), ('선생님', 'Noun'), ('께', 'Josa'), ('드릴', 'Noun'), ('꽃', 'Noun'), ('뭐', 'Noun'), ('가', 'Josa'), ('좋을까', 'Adjective'), ('?', 'Punctuation'), ('요즘', 'Noun'), ('가을', 'Noun'), ('이야', 'Josa'), ('.', 'Punctuation')]
Text: 친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?, POS Tags: [('친구', 'Noun'), ('가', 'Josa'), ('5월', 'Number'), ('봄', 'Noun'), ('에', 'Josa'), ('결혼', 'Noun'), ('하는데', 'Verb'), ('결혼식', 'Noun'), ('에', 'Josa'), ('줄', 'Noun'), ('꽃', 'Noun'), ('뭐', 'Noun'), ('가', 'Josa'), ('좋을까', 'Adjective'), ('?', 'Punctuation')]
Text: 응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해., POS Tags: [('응원', 'Noun'), ('의', 'Josa'), ('꽃', 'Noun'), ('좀', 'Noun'), ('추천', 'Noun'), ('해줘', 'Verb'), ('.', 'Punctuation'), ('특히', 'Adverb'), ('분홍', 'Noun'), ('꽃', 'Noun'), ('을', 'Josa'), ('좋아해', 'Adjective'), ('.', 'Punctuation')]
Text: 발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?, POS Tags: [('발렌타인데이', 'Noun'), ('겨울', 'Noun'), ('에', 'Josa'), ('여자친구', 'Nou

C:\Users\pc\AppData\Local\Temp\ipykernel_6668\3779665020.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['유사도'] = filtered_df.apply(lambda row: cosine_similarity(user_vector, np.array(row['최종_벡터']).reshape(1, -1))[0][0] * apply_event_weight_for_row(user_input, row), axis=1)
C:\Users\pc\AppData\Local\Temp\ipykernel_6668\3779665020.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['유사도'] = filtered_df['유사도'].astype(float)  # 숫자형으로 변환


Text: 친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?, POS Tags: [('친구', 'Noun'), ('가', 'Josa'), ('2월', 'Number'), ('에', 'Foreign'), ('졸업', 'Noun'), ('을', 'Josa'), ('하는데', 'Verb'), ('졸업식', 'Noun'), ('꽃', 'Noun'), ('추천', 'Noun'), ('해줄래', 'Verb'), ('?', 'Punctuation')]
Text: 사랑을 전할 꽃 추천해줘. 특히 빨강 꽃을 좋아해., POS Tags: [('사랑', 'Noun'), ('을', 'Josa'), ('전', 'Noun'), ('할', 'Verb'), ('꽃', 'Noun'), ('추천', 'Noun'), ('해줘', 'Verb'), ('.', 'Punctuation'), ('특히', 'Adverb'), ('빨강', 'Noun'), ('꽃', 'Noun'), ('을', 'Josa'), ('좋아해', 'Adjective'), ('.', 'Punctuation')]
Text: 크리스마스에 선물할 수 있는 꽃을 추천해줘., POS Tags: [('크리스마스', 'Noun'), ('에', 'Josa'), ('선물', 'Noun'), ('할', 'Verb'), ('수', 'Noun'), ('있는', 'Adjective'), ('꽃', 'Noun'), ('을', 'Josa'), ('추천', 'Noun'), ('해줘', 'Verb'), ('.', 'Punctuation')]


C:\Users\pc\AppData\Local\Temp\ipykernel_6668\3779665020.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['유사도'] = filtered_df.apply(lambda row: cosine_similarity(user_vector, np.array(row['최종_벡터']).reshape(1, -1))[0][0] * apply_event_weight_for_row(user_input, row), axis=1)
C:\Users\pc\AppData\Local\Temp\ipykernel_6668\3779665020.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['유사도'] = filtered_df['유사도'].astype(float)  # 숫자형으로 변환


Text: 하얀 꽃 추천해줘. 이번 달은 4월이야., POS Tags: [('하얀', 'Adjective'), ('꽃', 'Noun'), ('추천', 'Noun'), ('해줘', 'Verb'), ('.', 'Punctuation'), ('이번', 'Noun'), ('달', 'Noun'), ('은', 'Josa'), ('4월', 'Number'), ('이야', 'Foreign'), ('.', 'Punctuation')]
Text: 자원봉사자들에게 겨울에 감사의 마음을 전할 꽃은 무엇이 좋을까?, POS Tags: [('자원봉사자', 'Noun'), ('들', 'Suffix'), ('에게', 'Josa'), ('겨울', 'Noun'), ('에', 'Josa'), ('감사', 'Noun'), ('의', 'Josa'), ('마음', 'Noun'), ('을', 'Josa'), ('전', 'Noun'), ('할', 'Verb'), ('꽃', 'Noun'), ('은', 'Josa'), ('무엇', 'Noun'), ('이', 'Josa'), ('좋을까', 'Adjective'), ('?', 'Punctuation')]


...

C:\Users\pc\AppData\Local\Temp\ipykernel_6668\3779665020.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['유사도'] = filtered_df.apply(lambda row: cosine_similarity(user_vector, np.array(row['최종_벡터']).reshape(1, -1))[0][0] * apply_event_weight_for_row(user_input, row), axis=1)
C:\Users\pc\AppData\Local\Temp\ipykernel_6668\3779665020.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['유사도'] = filtered_df['유사도'].astype(float)  # 숫자형으로 변환


Text: 조카의 돌잔치 봄에 줄 꽃을 추천해 주세요., POS Tags: [('조카', 'Noun'), ('의', 'Josa'), ('돌잔치', 'Noun'), ('봄', 'Noun'), ('에', 'Josa'), ('줄', 'Noun'), ('꽃', 'Noun'), ('을', 'Josa'), ('추천', 'Noun'), ('해', 'Verb'), ('주세요', 'Verb'), ('.', 'Punctuation')]
Text: 직장 상사에게 5월에 드릴 꽃은 어떤 게 좋을까요?, POS Tags: [('직장', 'Noun'), ('상사', 'Noun'), ('에게', 'Josa'), ('5월', 'Number'), ('에', 'Foreign'), ('드릴', 'Noun'), ('꽃', 'Noun'), ('은', 'Josa'), ('어떤', 'Adjective'), ('게', 'Noun'), ('좋을까요', 'Adjective'), ('?', 'Punctuation')]
Text: 연말 12월 파티에 가져갈 꽃을 추천해 주세요., POS Tags: [('연말', 'Noun'), ('12월', 'Number'), ('파티', 'Noun'), ('에', 'Josa'), ('가져갈', 'Verb'), ('꽃', 'Noun'), ('을', 'Josa'), ('추천', 'Noun'), ('해', 'Verb'), ('주세요', 'Verb'), ('.', 'Punctuation')]
Text: 아빠가 8월에 정년 퇴임하시는데 드릴 꽃을 추천해줄래?, POS Tags: [('아빠', 'Noun'), ('가', 'Josa'), ('8월', 'Number'), ('에', 'Foreign'), ('정년', 'Noun'), ('퇴', 'Noun'), ('임하시는데', 'Verb'), ('드릴', 'Noun'), ('꽃', 'Noun'), ('을', 'Josa'), ('추천', 'Noun'), ('해줄래', 'Verb'), ('?', 'Punctuation')]
Text: 10월 가

C:\Users\pc\AppData\Local\Temp\ipykernel_6668\3779665020.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['유사도'] = filtered_df.apply(lambda row: cosine_similarity(user_vector, np.array(row['최종_벡터']).reshape(1, -1))[0][0] * apply_event_weight_for_row(user_input, row), axis=1)
C:\Users\pc\AppData\Local\Temp\ipykernel_6668\3779665020.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['유사도'] = filtered_df['유사도'].astype(float)  # 숫자형으로 변환


Text: 할머니 생신 겨울에 드릴 꽃은 어떤 게 좋을까요?, POS Tags: [('할머니', 'Noun'), ('생신', 'Noun'), ('겨울', 'Noun'), ('에', 'Josa'), ('드릴', 'Noun'), ('꽃', 'Noun'), ('은', 'Josa'), ('어떤', 'Adjective'), ('게', 'Noun'), ('좋을까요', 'Adjective'), ('?', 'Punctuation')]
Text: 할아버지께 8월에 드릴 꽃을 추천받고 싶어요., POS Tags: [('할아버지', 'Noun'), ('께', 'Josa'), ('8월', 'Number'), ('에', 'Foreign'), ('드릴', 'Noun'), ('꽃', 'Noun'), ('을', 'Josa'), ('추천', 'Noun'), ('받고', 'Verb'), ('싶어요', 'Verb'), ('.', 'Punctuation')]
Text: 가정의 달인 5월 봄에 가족에게 꽃을 선물하고 싶습니다., POS Tags: [('가정', 'Noun'), ('의', 'Josa'), ('달인', 'Noun'), ('5월', 'Number'), ('봄', 'Noun'), ('에', 'Josa'), ('가족', 'Noun'), ('에게', 'Josa'), ('꽃', 'Noun'), ('을', 'Josa'), ('선물', 'Noun'), ('하고', 'Josa'), ('싶습니다', 'Verb'), ('.', 'Punctuation')]
Text: 평소에 고마움을 전하고 싶은 사람에게 가을에 선물할 꽃을 추천받고 싶어., POS Tags: [('평소', 'Noun'), ('에', 'Josa'), ('고마움', 'Noun'), ('을', 'Josa'), ('전하', 'Noun'), ('고', 'Josa'), ('싶은', 'Verb'), ('사람', 'Noun'), ('에게', 'Josa'), ('가을', 'Noun'), ('에', 'Josa'), ('선물', 'Noun'), ('할', 

,user_input,추천꽃1,추천꽃2,추천꽃3
0,스승의 날에 선생님께 드릴 꽃 뭐가 좋을까? 요즘 가을이야.,과꽃,수크령,매화
1,친구가 5월 봄에 결혼하는데 결혼식에 줄 꽃 뭐가 좋을까?,무릇,제비고깔,겹벗꽃
2,응원의 꽃 좀 추천해줘. 특히 분홍 꽃을 좋아해.,부용,인게이지먼트 장미,큐티버블 소국
3,발렌타인데이 겨울에 여자친구에게 줄 꽃 뭐가 좋을까?,인챈트먼트 장미,장미,풀몬티 장미
4,친구가 2월에 졸업을 하는데 졸업식 꽃 추천해줄래?,아도니스,네모필라,살구꽃
...,...,...,...,...
95,5월 봄 결혼식 축하 화환으로 어떤 꽃이 좋을까요?,카네이션,덴파레,데이지
96,오래된 연인에게 특별한 날에 봄에 기념할 꽃은 무엇이 있니?,보라색라일락,아도니스,라일락
97,직장 동료에게 감사 편지와 함께 보낼 가을의 꽃을 추천해줘.,수크령,메리골드,흰독말풀
98,공부에 지친 친구에게 10월에 줄 꽃을 추천받고 싶어요.,이끼장미,매화,흰독말풀


In [64]:
df_results.to_csv("C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/klue_테스트_모델결과(수정 후).csv")